In [53]:
import pandas as pd
import numpy as np
import os
files = os.listdir("../data")

In [54]:
def fix_array(x):
    x = np.fromstring(
    x.replace('\n','')
    .replace('[','')
    .replace(']','')
    .replace('  ',' '), sep=' ')
    return x.reshape((1, 768))

In [55]:
qa = pd.read_csv("../data/" + files[0])
for file in files[1:]:
    print(file)
    qa = pd.concat([qa, pd.read_csv("../data/" + file)], axis = 0)
    

healthtapQAs.csv
questionDoctorQAs.csv
webmdQAs.csv
ehealthforumQAs.csv


In [56]:
qa["question_bert"] = qa["question_bert"].apply(fix_array)
qa["answer_bert"] = qa["answer_bert"].apply(fix_array)

In [57]:
qa = qa.sample(frac = 1)
qa.reset_index(inplace = True, drop = True)

In [58]:
question_bert = np.concatenate(qa["question_bert"].values, axis=0)
answer_bert = np.concatenate(qa["answer_bert"].values, axis=0)

In [59]:
answer_bert.shape

(166804, 768)

In [60]:
question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')

In [62]:
import faiss
answer_index = faiss.IndexFlatL2(answer_bert.shape[-1])
answer_index.add(answer_bert)

question_index = faiss.IndexFlatL2(question_bert.shape[-1])
question_index.add(question_bert)

In [137]:
k = len(question_bert)
n_items = 1000
D1, I1 = answer_index.search(question_bert[0:1].astype('float32'), k)
D2, I2 = question_index.search(question_bert[0:1].astype('float32'), k)

In [138]:
closest_ind_q = list(I2[0, :n_items]) + list(I2[0, -n_items:])
closest_ind_a = list(I1[0, :n_items]) + list(I1[0, -n_items:])

In [139]:
dist_answers = answer_bert[closest_ind_a, :]
dist_questions = question_bert[closest_ind_q, :]

In [158]:
from sklearn.decomposition import TruncatedSVD
from MulticoreTSNE import MulticoreTSNE as TSNE
import umap
reducer = TSNE(n_components = 3, perplexity=30, n_iter = 5000)

In [159]:
reduced_dimensions = reducer.fit_transform(np.concatenate([dist_questions, dist_answers], axis = 0))

In [160]:
question_bert_3d_close = reduced_dimensions[:n_items]
question_bert_3d_far = reduced_dimensions[n_items:n_items*2]
answer_bert_3d_close = reduced_dimensions[n_items*2:n_items*3]
answer_bert_3d_far = reduced_dimensions[n_items*3:]

In [164]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import numpy as np
orig_point = go.Scatter3d(
    name = "Original Question",
    x=question_bert_3d_close[0:1,0],
    y=question_bert_3d_close[0:1,1],
    z=question_bert_3d_close[0:1,2],
    mode='markers',
    text = qa["question"].loc[closest_ind_q[:1]],
    marker=dict(
        size=12,
        line=dict(
            color='rgba(255, 0, 0, 0.14)',
            width=0.1
        ),
        opacity=1.0
    )
)

close_q = go.Scatter3d(
    name = "Similar Questions",
    x=question_bert_3d_close[:,0],
    y=question_bert_3d_close[:,1],
    z=question_bert_3d_close[:,2],
    mode='markers',
    text = qa["question"].loc[closest_ind_q],
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.1
        ),
        opacity=0.8
    )
)

close_a = go.Scatter3d(
    name = "Similar Answers",
    x=answer_bert_3d_close[:,0],
    y=answer_bert_3d_close[:,1],
    z=answer_bert_3d_close[:,2],
    mode='markers',
    text = qa["question"].loc[closest_ind_a],
    marker=dict(
        size=4,
        line=dict(
            color='rgba(244, 100, 40, 0.14)',
            width=0.1
        ),
        opacity=0.8
    )
)

far_q = go.Scatter3d(
    name = "Dissimilar Questions",
    x=question_bert_3d_far[:,0],
    y=question_bert_3d_far[:,1],
    z=question_bert_3d_far[:,2],
    mode='markers',
    text = qa["question"].loc[closest_ind_q],
    marker=dict(
        size=4,
        line=dict(
            color='rgba(40, 100, 217, 0.14)',
            width=0.1
        ),
        opacity=0.8
    )
)

far_a = go.Scatter3d(
    name = "Dissimilar Answers",
    x=answer_bert_3d_far[:,0],
    y=answer_bert_3d_far[:,1],
    z=answer_bert_3d_far[:,2],
    mode='markers',
    text = qa["question"].loc[closest_ind_a],
    marker=dict(
        size=4,
        line=dict(
            color='rgba(255, 40, 40, 0.14)',
            width=0.1
        ),
        opacity=0.8
    )
)

data = [orig_point, close_q, close_a, far_q, far_a]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')

In [157]:
from plotly import offline
offline.plot(fig, filename='file.html')

'file:///Users/ryanchesler/Documents/tf-hackathon/AskDoc_Experiments/notebooks/file.html'